# MSA 2023 Phase 2 - Deploy Model Endpoint

### 1. Load necessary packages and modules

In [1]:
# Azure Machine Learning SDK core
from azureml.core import Workspace
from azureml.core.model import Model

### 2. Load and connect to workspace

In [2]:
ws = Workspace.from_config(path="config.json")

### 3. Register model onto Azure

In [3]:
model = Model.register(
    ws,
    model_name="segmentation-logistic-regression",
    model_path="../2. Training and Evaluation/outputs/lg-model.pkl"
)

Registering model segmentation-logistic-regression


### 4. Create entry script

In [4]:
%%writefile score.py
import json, os, joblib
import numpy as np

# Since the model works with label-encoded data, we can create a dictionary to get the actual class names
classes = {"A": "A", "B": "B", "C": "C", "D": "D"}

def init():
    # Loads the model
    global model
    model_path = "../2. Training and Evaluation/outputs/lg-model.pkl"
    full_model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), model_path)
    model = joblib.load(full_model_path)

def run(request):
    # Loads the input data, runs the model on it, and returns its predictions
    data = json.loads(request)
    data = np.array(data["data"])
    result = model.predict(data)
    return result.tolist()

Overwriting score.py


### 5. Test and share endpoint

In [6]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {"data": [[1]*22,[0]*22]}

body = str.encode(json.dumps(data))

url = 'https://msa-phase2-azure-flybu.australiaeast.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'y105ZKDmFTMNjviEATyqeWoYqvfjMqWE'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'segmentation-logistic-regression' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'["B", "D"]'
